In [13]:
#importing required libraries
import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys

import glob
from imageai.Detection import VideoObjectDetection
import natsort
import numpy as np
import os
import sys
import numpy as np
import cv2
import glob
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, nan, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint



In [15]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , "C:/RoadCrossingAssistant/Data/resnet50_coco_best_v2.0.1.h5"))
detector.loadModel()
custom_objects = detector.CustomObjects(bicycle=True, motorcycle=True,car=True,truck=True)
detections = detector.detectCustomObjectsFromImage(custom_objects = custom_objects,input_image=os.path.join(execution_path , "C:/RoadCrossingAssistant/Data/Frames/video1/frame20.jpg"), output_image_path=os.path.join(execution_path , "image2new.jpg"), minimum_percentage_probability=40)



for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

car  :  42.50149130821228  :  [1397, 494, 1479, 588]
--------------------------------
car  :  57.264167070388794  :  [1411, 490, 1563, 631]
--------------------------------
truck  :  52.60764956474304  :  [30, 360, 323, 527]
--------------------------------
car  :  55.52141070365906  :  [41, 358, 329, 525]
--------------------------------
motorcycle  :  79.2188286781311  :  [1, 460, 193, 647]
--------------------------------
motorcycle  :  83.647620677948  :  [1056, 529, 1230, 696]
--------------------------------
motorcycle  :  88.93356323242188  :  [387, 547, 795, 839]
--------------------------------
truck  :  62.59300112724304  :  [516, 310, 1058, 608]
--------------------------------
car  :  73.24953079223633  :  [1677, 570, 1900, 1046]
--------------------------------


In [16]:
bboxes = []
            
for i in range(len(detections)):
    bboxes.append(list(detections[i]['box_points']))
                
print(bboxes)

[[1397, 494, 1479, 588], [1411, 490, 1563, 631], [30, 360, 323, 527], [41, 358, 329, 525], [1, 460, 193, 647], [1056, 529, 1230, 696], [387, 547, 795, 839], [516, 310, 1058, 608], [1677, 570, 1900, 1046]]


In [17]:
x = np.load("C:/RoadCrossingAssistant/Data/Arrays_RetinaNet/array1.npy", allow_pickle=True)
print(x[20])

[[1396, 488, 1516, 590], [1412, 490, 1562, 625], [1057, 527, 1232, 699], [39, 357, 326, 527], [43, 358, 328, 526], [4, 439, 193, 648], [1625, 550, 1739, 753], [1476, 555, 1734, 759], [530, 309, 1053, 597], [386, 485, 793, 798], [1676, 630, 1902, 1041]]


In [18]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker


def get_direction(videoPath, trackerType, boxes_list, frame_no):

    '''
    Detects direction of each vehicle in the video and returns and a list 
    containing these directions

    Parameters:
    videoPath(string) : path of the video
    trackerType(string) : the openCV tracker type to be used  
    boxes_list(list) : list containing bounding boxes for all detected vehicles 
    frame_no : frame no of the video for which you require directions

    '''
    k = 0 #iterator for frame no
    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    directions = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - 5

    #reading the video
    cap = cv2.VideoCapture(videoPath)
    success, frame = cap.read()
    if not success:
        print('Failed to read video')
        sys.exit(1)

    #the direction detection logic starts here
    while cap.isOpened() :
        success, frame = cap.read()
        if not success:
            break
        
        #saving past few frames as we will apply traking in backward direction
        #form the frame_no
        if( k == frame_no -10 ):
            frame_b10 = frame
        if ( k == frame_no - 9):
            frame_b9 = frame
        if ( k == frame_no - 8):
            frame_b8 = frame
        if ( k == frame_no - 7):
            frame_b7 = frame
        if ( k == frame_no - 6):
            frame_b6 = frame
        if ( k == frame_no - 5):
            frame_b5 = frame
        if ( k == frame_no - 4):
            frame_b4 = frame
        if ( k == frame_no - 3):
            frame_b3 = frame
        if ( k == frame_no - 2):
            frame_b2 = frame
        if ( k == frame_no - 1):
            frame_b1 = frame

        if k == frame_no:
            #boxes_list = [tuple(l) for l in boxes_list]
            for i in range(n):
                box = boxes_list[i]
                x1[i] = (box[0]+2*box[2])/2
                multiTracker_back.add(createTrackerByName(trackerType), frame, box)
            
            #all the vehicles from boxes_list added in the traker instance
            #applying vehicle tracking in backward direction
                
            (success_back, boxes_back) = multiTracker_back.update(frame_b1)
            (success_back, boxes_back) = multiTracker_back.update(frame_b2)
            (success_back, boxes_back) = multiTracker_back.update(frame_b3)
            (success_back, boxes_back) = multiTracker_back.update(frame_b4)
            (success_back, boxes_back) = multiTracker_back.update(frame_b5)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b6)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b7)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b8)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b9)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b10)



            for i in range(n):
                    b = boxes_back[i]
                    x2[i] = (b[0]+2*b[2])/2
                    #print('x1',i,x1[i])


            for i in range(n):
                
                x_d =  x2[i] - x1[i]
                #print(x_d)
                if x_d > 0:
                    directions[i] = 1
                elif x_d < 0:
                    directions[i] = -1

            for i in range(n):
                box = boxes_list[i]
                (x,y,w,h) = [int(v) for v in box]
                if(directions[i] == 0): # not moving
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                elif(directions[i] == 1): # direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
                elif(directions[i] == -1): # opposite to direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)

            #cv2.imshow('frame', frame)
            cv2.waitKey(0)
            break
        k = k+1
        
    cv2.destroyAllWindows()
    return np.array(directions)

In [20]:
boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in bboxes]

c = get_direction("C:/RoadCrossingAssistant/Data/Videos/video1.MOV", "KCF", boxes_list, 20)

print(c)

#x = np.load("C:/RoadCrossingAssistant/Data/Directions_RetinaNet/directions1.npy", allow_pickle=True)
#print(x[20])

[ 1  1 -1 -1 -1  1  1 -1 -1]
[ 1  1  1 -1 -1 -1  1  1 -1  1 -1]


In [22]:
def get_speeds_from_frame(video, boxes_list, frame_no, no_previous_frames, trackerType = "KCF"):

    '''
    Detects speed of each vehicle in the frame and returns and an containing these speeds

    Parameters:
    video(string) : path of the video
    frame_no : frame no of the video for which you require directions
    boxes_list(list) : list containing bounding boxes for all detected vehicles in the particular frame
    no_previous_frames(int) : number of previous frames to be considered while calculating the speed
    trackerType(string) : the openCV tracker type to be used

    '''
    frame_buffer = []
    #cv2.namedWindow(video, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(video, height=650, width=1156)
    cap = cv2.VideoCapture(video)

    for i in range(frame_no - no_previous_frames, frame_no + 1):

        cap.set(1,i)
        ret, frame = cap.read()
        frame_buffer.append(frame)

    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    speeds = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    y1 = [0]*n #y coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - no_previous_frames
    y2 = [0]*n #y coordinates of vehicles in frame_no - no_previous_frames

    frame = frame_buffer.pop()
    for i in range(n):
        box = boxes_list[i]
        x1[i] = (box[0]+2*box[2])/2
        y1[i] = (box[1]+2*box[3])/2
        multiTracker_back.add(createTrackerByName(trackerType), frame, box)
        #all the vehicles from boxes_list added in the traker instance
        #applying vehicle tracking in backward direction
    for j in range(no_previous_frames):
        frame = frame_buffer.pop()
        (success_back, boxes_back) = multiTracker_back.update(frame)

    for i in range(n):
        b = boxes_back[i]
        x2[i] = (b[0]+2*b[2])/2
        y2[i] = (b[1]+2*b[3])/2
        x_d = x2[i] - x1[i]
        y_d = y2[i] - y1[i]
        dist = (x_d**2 + y_d**2)**0.5
        speeds[i] = dist

    return np.array(speeds)

In [23]:
s = get_speeds_from_frame("C:/RoadCrossingAssistant/Data/Videos/video1.MOV", boxes_list, 20, 6)

print(s)

[ 12.          10.11187421   3.53553391   4.03112887  37.85498646
  37.27264412 101.00495037  66.48308055   3.35410197]
